In [14]:
#Jesse Hazard
#Pawel Linek

import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.linear = nn.Linear(3072, 10)
        self.fc3 = nn.Linear(10, 4)

    def forward(self, x):
        x = x.view(4, 3*32*32)
        x = self.linear(x).clamp(min=0)
        x = self.fc3(x)
   
        return x
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=4,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=4,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')
net = Net()

running_loss = 0.0
optimizer = torch.optim.SGD(net.parameters(), lr=0.001, momentum = 0.9)
for epoch in range(5):  # loop over the dataset multiple times

    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, y = data
        # forward + backward + optimize
        outputs = net(inputs)
        target = torch.ones(4)
        loss = torch.nn.MarginRankingLoss(margin=1).forward(outputs,y,target)
        # forward + backward + optimize
       
        loss.backward()
        optimizer.step()          
        optimizer.zero_grad()
       
        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0
           
print('Finished Training')
PATH = './cifar_net.pth'
torch.save(net.state_dict(), PATH)



Files already downloaded and verified
Files already downloaded and verified
[1,  2000] loss: 0.390
[1,  4000] loss: 0.028
[1,  6000] loss: 0.018
[1,  8000] loss: 0.014
[1, 10000] loss: 0.010
[1, 12000] loss: 0.007
[2,  2000] loss: 0.007
[2,  4000] loss: 0.003
[2,  6000] loss: 0.003
[2,  8000] loss: 0.003
[2, 10000] loss: 0.003
[2, 12000] loss: 0.003
[3,  2000] loss: 0.002
[3,  4000] loss: 0.001
[3,  6000] loss: 0.001
[3,  8000] loss: 0.002
[3, 10000] loss: 0.001
[3, 12000] loss: 0.001
[4,  2000] loss: 0.001
[4,  4000] loss: 0.001
[4,  6000] loss: 0.001
[4,  8000] loss: 0.000
[4, 10000] loss: 0.001
[4, 12000] loss: 0.001
[5,  2000] loss: 0.000
[5,  4000] loss: 0.001
[5,  6000] loss: 0.000
[5,  8000] loss: 0.000
[5, 10000] loss: 0.000
[5, 12000] loss: 0.001
Finished Training


In [1]:
dataiter = iter(testloader)
images, labels = dataiter.next()
net = Net()
net.load_state_dict(torch.load(PATH))
outputs = net(images)
_, predicted = torch.max(outputs, 1)

print('Predicted: ', ' '.join('%5s' % classes[predicted[j]]
                              for j in range(4)))
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 10000 test images: %d %%' % (
    100 * correct / total))
class_correct = list(0. for i in range(10))
class_total = list(0. for i in range(10))
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = net(images)
        _, predicted = torch.max(outputs, 1)
        c = (predicted == labels).squeeze()
        for i in range(4):
            label = labels[i]
            class_correct[label] += c[i].item()
            class_total[label] += 1


for i in range(10):
    print('Accuracy of %5s : %2d %%' % (
        classes[i], 100 * class_correct[i] / class_total[i]))

NameError: name 'testloader' is not defined